In [72]:
from langchain.docstore.document import Document
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter, SpacyTextSplitter, TextSplitter
from langchain.vectorstores import Chroma
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.document_loaders import UnstructuredPDFLoader, PyMuPDFLoader
from langchain.indexes.vectorstore import VectorstoreIndexCreator

In [73]:
import os
import openai
openai.api_key = "<API KEY GOES HERE>"
os.environ["OPENAI_API_KEY"] = openai.api_key

In [74]:
loader = PyMuPDFLoader("../data/orig-documents/ashford-zoning-code.pdf")

In [75]:
from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator().from_loaders([loader])

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.
Exiting: Cleaning up .chroma directory


In [76]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [77]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="map_reduce")

In [78]:
query = """
What are the zoning districts defined in this document? Return your results as a JSON array in the form [{ "district_name": <district name>, "district_abbr": <district abbreviation> }]."""

In [82]:
docs = index.vectorstore.similarity_search(query, k=10)

In [83]:
docs

[Document(page_content='the district boundary line shall be determined using the scale on the Official Zoning Map. \n \nC. Where a District boundary divides a lot in one ownership into a residential and a nonresidential District  or into two \nnonresidential Districts, the area and frontage requirements for that lot shall comply with those that are more \nrestrictive as set forth for such Districts. All other building requirements shall correspond with those of the particular \nDistrict in which a use, structure or building is established or constructed. \n \nD. Where the location of a district boundary line is uncertain, the Commission shall interpret the district boundary line.', lookup_str='', metadata={'file_path': '../data/orig-documents/ashford-zoning-code.pdf', 'page_number': 21, 'total_pages': 102, 'format': 'PDF 1.5', 'title': '', 'author': 'DFamiglietti', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word 2010', 'producer': 'Microsoft® Word 2010', 'creationDate': "D:2

In [84]:
chain.run(input_documents=docs, question=query)

' [{ "district_name": "Residential - Agricultural", "district_abbr": "RA" }, { "district_name": "General Commercial", "district_abbr": "C" }, { "district_name": "Interstate Interchange Development", "district_abbr": "SD" }, { "district_name": "Technology Development", "district_abbr": "TD" }]'